In [49]:
#! /usr/bin/env python
# antcat_api - antcat.org API inteface sandbox

import requests
import json

# this gest first 100 reference records as a single json string
r = requests.get('http://antcat.org/v1/references')

# this gets first 100 journals
jr = requests.get('http://antcat.org/v1/journals')

# this gets first 100 publishers
pr = requests.get('http://antcat.org/v1/publishers')

refs = r.text
journals = jr.text
publishers = pr.text

# parse json
parsed_refs = json.loads(refs)
parsed_journals = json.loads(journals)
parsed_publish = json.loads(publishers)


# get a list of dicts where reference type is one of the elements
new_journals = []
for rec in parsed_journals:
    rec_dict = list(rec.values())[0]
    rec_type = list(rec.keys())[0]
    rec_dict['type'] = rec_type
    new_journals.append(rec_dict)
    
# get a list of 1-dimensional dicts for publishers
new_publishers = []
for publish in parsed_publish:
    publish_dict = list(publish.values())[0]
    new_publishers.append(publish_dict)  
    
def get_jour_name(id):
    for journal in new_journals:
        if journal['id'] == id:
            name = journal['name']
            return name
        
def get_publ_name(id):
    for publisher in new_publishers:
        if publisher['id'] == id:
            name = publisher['name']
            return name
        
def get_bib_dict(reference):
    ref_type = list(reference.keys())[0]
    ref_dict = list(reference.values())[0]
    #print(ref_dict)
    bib_dict = {'ref_type': ref_type, 
               'author' : ref_dict['author_names_string_cache'],
               'title' : ref_dict['title'],
               'year' : ref_dict['year'],
               'journal' : get_jour_name(ref_dict['journal_id']),
               'ser_vol_iss' : ref_dict['series_volume_issue'],
               'publisher' : get_publ_name(ref_dict['publisher_id']),
               'pages' : ref_dict['pagination']}
    return bib_dict

def print_bib_str(bib_dict):
    if bib_dict['ref_type'] == 'article_reference':
        ref_type = 'article'
    elif bib_dict['ref_type'] == 'book_reference':
        ref_type = 'book'
    elif bib_dict['ref_type'] == 'nested_reference':
        ref_type = 'misc'
        
    first_author = bib_dict['author'].split(' ')[0]
    first_author = first_author.translate({ord(','): ''})
    all_authors = bib_dict['author'].translate({ord(';'): ' and'})
    year = bib_dict['year']
    title = bib_dict['title']
    title_words = title.split(' ')
    new_title_words = []
    for word in title_words:
        if word[0].isupper():
            word = '{{{}}}'.format(word)
        elif word[0] == '(' and word[1].isupper():
            word = '{{{}}}'.format(word)
        elif '*' in word:
            word = word.replace('*', '')
            word = '\\textit{{{}}}'.format(word)
        new_title_words.append(word)
    title = ' '.join(new_title_words)
    keyword = ('{}_{}'.format(first_author, year))
    journal = bib_dict['journal']
    if bib_dict['ser_vol_iss']:
        split = bib_dict['ser_vol_iss'].split('(')
        volume = split[0]
        if len(split) >= 2:
            issue = split[1].replace(')', '')
    pages = bib_dict['pages']
    publisher = bib_dict['publisher']
    
    print('@{}{{{},'.format(ref_type, keyword))
    print('  author = {{{}}},'.format(all_authors))
    print('  title = {{{}}},'.format(title))
    print('  year = {{{}}},'.format(year))
    if ref_type == 'article':
        print('  journal = {{{}}},'.format(journal))
        print('  volume = {{{}}},'.format(volume))
        if len(split) >= 2:
            print('  issue = {{{}}},'.format(issue))
    if ref_type == 'book':
        # how to guarantee that the last element won't have comma?
        print('  publisher = {{{}}},'.format(publisher))
    if pages:
        print('  pages = {{{}}}'.format(pages))
    print('}')

    

# the parsed contents are stored as a list of nested dicts
#print(parsed_refs)

for ref in parsed_refs:
    #print(ref)
    bibd = get_bib_dict(ref)
    print_bib_str(bibd)

@article{Abdul-Rassoul_1978,
  author = {Abdul-Rassoul, M. S. and Dawah, H. A. and Othman, N. Y.},
  title = {{Records} of insect collection {(Part} {I)} in the {Natural} {History} {Research} {Centre,} {Baghdad}},
  year = {1978},
  journal = {Bulletin of the Natural History Research Centre. University of Baghdad},
  volume = {7},
  issue = {2},
  pages = {1-6}
}
@article{Abe_1991,
  author = {Abe, M. and Smith, D. R.},
  title = {{The} genus-group names of {Symphyta} {(Hymenoptera)} and their type species},
  year = {1991},
  journal = {Esakia},
  volume = {31},
  pages = {1-115}
}
@book{Ikehara_1974,
  author = {Ikehara, S. (ed.)},
  title = {{Ecological} studies of nature conservation of the {Ryukyu} {Islands} (1). {Report} for the fiscal year of 1973. [In {Japanese.]}},
  year = {1974},
  publisher = {University of the Ryukyus},
  pages = {278 pp.}
}
@misc{Abe_1974,
  author = {Abe, T.},
  title = {{Notes} on the fauna of ants in {Iriomote} {Island.} [In {Japanese.]}},
  year = {19